# Suspicious Execution from Outlook
Detects EnableUnsafeClientMailRules used for Script Execution from Outlook

## Rule Content
```
- title: Suspicious Execution from Outlook
  id: e212d415-0e93-435f-9e1a-f29005bb4723
  status: experimental
  description: Detects EnableUnsafeClientMailRules used for Script Execution from
    Outlook
  references:
  - https://github.com/sensepost/ruler
  - https://www.fireeye.com/blog/threat-research/2018/12/overruled-containing-a-potentially-destructive-adversary.html
  tags:
  - attack.execution
  - attack.t1059
  - attack.t1202
  author: Markus Neis
  date: 2018/12/27
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    clientMailRules:
      CommandLine: '*EnableUnsafeClientMailRules*'
    outlookExec:
      ParentImage: '*\outlook.exe'
      CommandLine: \\\\*\\*.exe
    condition: clientMailRules or outlookExec
  falsepositives:
  - unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_command_line.keyword:*EnableUnsafeClientMailRules* OR (process_parent_path.keyword:*\\outlook.exe AND process_command_line.keyword:\\\\*\\*.exe))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()